### Ben Ezra 316046374
### Amit Nahum 315871400
# PLEASE READ!
- ## If you want to exit the chatbot write exit.

### Intents – Separated to 2: Primary and sub-intent __(Primary, sub-intent)__:
1.	Book, Hotel.
2.	Book, car.
3.	Book, flight.
4.	Book, package.
5.	Change, location
6.	Change, date
7.	Cancel.
8.	Exit.
### Main States of chatbot (State_machine):
1.	Online.
2.	Offline.
### Conversation flow:
#### Initiating the chatbot/exiting  Choosing state (offline/online/exiting)  Asking for user’s name  (0) asking for user’s intents (“What can I help you?”).
1. __ If primary intent = book:__
-  (1 - "Book Hotel") secondary Intent = Hotel:
(2 "Get location") Asking for location  (3 "Get date") asking for dates (start and end)  registering to csv file and giving booking id.

-	(4 "Book Car") If secondary Intent = car:
(2 "Get location") Asking for location  (3 "Get date") asking for dates (start and end)   registering to csv file and giving booking id.

-	(5 "Book Flight") If secondary Intent = flight:
(2 "Get location") Asking for location  (3 "Get date") asking for dates (start and end)  registering to csv file and giving booking id.

-	(6 "Book Package") If secondary Intent = package:
(2 "Get location") Asking for location  (3 "Get date") asking for dates (start and end)  registering to csv file and giving booking id.

2.	__(7 "Change" ) If primary intent = change:__
-	 If secondary intent = location:
(8 "Get ID") Asking for booking ID  (7 "Change") asking for attribute to change (location, dates)  (2 "Get location") Asking for location  saving in database and giving confirmation output  asking If more help needed (if yes, returning to ask the user’s intent, if no further help needed, exiting).

-  If secondary intent = change date:
(8 "Get ID") Asking for booking ID  (7 "Change") asking for attribute to change (location, dates)  (3 "Get date") Asking for date  saving in database and giving confirmation output   asking If more help needed (if yes, returning to ask the user’s intent, if no further help needed, exiting).

3. __("Cancel") If primary intent = cancel:__
(8 "Get ID") Asking for booking ID  canceling and updating database.

4. __("Exit") If primary intent = exit:__
Exiting the program.
5. __(-1) If primary intent = "Not Recognized":__
Informing the user intent not recognized.


- ### The DataFrame was created at the start using our minds to define sentences that the users may give the chatbot, then we started paraphrasing it using "quillbot" AI. The more sentences, the more accurate the model.
- ### The classifier is based on the same classifier we used in the middleHW. It's NaiveBayesClassifier. We had a struggle using it as multi-label classifier, so we managed to work through it and classify the data separately (for each intent - primary and secondary) and still get good results - accuracy 0.93.
- ### More explanations in the rest of the notebook.


# First, Installations and Imports!


In [37]:
import nltk
import re
import csv
import pandas as pd
import requests
import numpy as np
import uuid
import random
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import NaiveBayesClassifier
from datetime import date
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ezrab\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ezrab\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\ezrab\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ezrab\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# Chat main menus

In [38]:
def run_chat():
    #print("Welcome to the tourism Chatbot!"
    #     "\n Press number and then enter."
    #     "\n 1. To enter modes."
    #     "\n 2. To exit.")
    choice = input("Welcome to the Tourism Chatbot!"
                   "\n Press number and than enter."
                   "\n 1. To enter modes."
                   "\n 2. To exit.")
    choose = True
    while choose:
        if choice == "1":
            choose = False
            choose_state()
        elif choice == "2" or choice =="exit":
            print(f"exiting.")
            return
        else:
            print("Invalid selection. Please try again.")
def choose_state():
    #print("Please select a mode"
    #     "\n 1. Offline"
    #     "\n 2. Online"
    #    "\n 3. To exit."
    #     "\nEnter your selection (1 or 2 or 3): ")#

    choice = input("Please select a mode:"
                   "\n1. Offline"
                   "\n2. Online"
                   "\n3. Exit"
                   "\nEnter your selection (1 or 2 or 3): ")
    choose = True
    while choose:
        if choice == "1":
            print(f"Chatbot is now in Offline mode.")
            choose = False
            run_offline_chatbot()
        elif choice == "2":
            print(f"Chatbot is now in Online mode.")
            choose = False
            run_online_chatbot()
        elif str.lower(choice) == 'exit' or choice =="3":
            print(f"Chatbot is now closed.")
            choose = False
            exit()
    else:
        print("Invalid selection. Please try again.")

# Building 2 Classifiers, for primary intent and for sub intent.

We've built a function that runs on random states and finds the best random state to our data. running loop that runs on average accuracy of the 2 models.
At the end, we chose the index of the best random, it's number 68.
```
def find_best_random():
    avg_acc_list =[]
    for x in range(100):
        np.random.seed(x)
        # Load data from csv file
        data = pd.read_csv('data.csv')
        np.random.shuffle(data.values[1:, :])
        avg_acc = (prime_class(data)+sub_class(data))/2
        avg_acc_list.append(avg_acc)

    max_acc = max(avg_acc_list)
    index = avg_acc_list.index(max_acc)

    return avg_acc_list,max_acc,index
```
we changed a bit the classifies functions to run the loop (just defined them as function).

In [39]:
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # Get word frequency distribution
    freq_dist = FreqDist(filtered_tokens)
    return dict(freq_dist)

In [40]:
#Load data from csv file
np.random.seed(68)
data = pd.read_csv('data.csv')
np.random.shuffle(data.values[1:, :])

#### Primary intent classification

In [41]:
primary_data = data.drop('sub',axis=1)
# Apply feature extraction to the data
features_prime = [(preprocess_text(text), intent) for text, intent in zip(primary_data['text'], primary_data['primary'])]

# Split the data into training and testing sets
train_set_size_prime = int(len(features_prime) * 0.8)

train_set_prime = features_prime[:train_set_size_prime]
test_set_prime = features_prime[train_set_size_prime:]

# Train the classifier
classifier_prime = NaiveBayesClassifier.train(train_set_prime)

# Evaluate the classifier
accuracy_prime = nltk.classify.accuracy(classifier_prime, test_set_prime)
#print(accuracy_prime)

#### Sub intent classification

In [42]:
sub_data = data.drop('primary',axis=1)
# Apply feature extraction to the data
features_sub = [(preprocess_text(text), intent) for text, intent in zip(sub_data['text'], sub_data['sub'])]

# Split the data into training and testing sets
train_set_size_sub = int(len(features_sub) * 0.8)
train_set_sub = features_sub[:train_set_size_sub]
test_set_sub = features_sub[train_set_size_sub:]

# Train the classifier
classifier_sub = NaiveBayesClassifier.train(train_set_sub)

# Evaluate the classifier
accuracy_sub = nltk.classify.accuracy(classifier_sub, test_set_sub)
#print(accuracy_sub)

# Using different NLP methods to analyze the input

In [43]:
# Extracting the features of given user input.
def extract_features(text):
    words = word_tokenize(text)
    return dict([(word, True) for word in words])

# function to classify user input
def classify_input(text):
    features = extract_features(text)

    # Get primary intent from classifier_prime
    probs_prime = classifier_prime.prob_classify(features)
    primary_intent = max(probs_prime.samples(), key=lambda label: probs_prime.prob(label))

    # Get sub-intent from classifier_sub
    probs_sub = classifier_sub.prob_classify(features)
    sub_intent = max(probs_sub.samples(), key=lambda label: probs_sub.prob(label))

    return [[primary_intent, sub_intent]]
# function to track the sentiment of the conversation
def track_sentiment(text):
    # Use SentimentIntensityAnalyzer to track the sentiment of the conversation
    sid = SentimentIntensityAnalyzer()
    sentiment = sid.polarity_scores(text)
    return sentiment

# function to extract entities from user input
def extract_entities(text):
    # Tokenize the text
    words = word_tokenize(text)
    # POS tag the words
    pos_tagged_words = nltk.pos_tag(words)
    # Use NLTK's named entity chunker to extract entities
    entities = nltk.ne_chunk(pos_tagged_words)
    # Extract named entities that are not part of a subTree
    main_entities = []
    for entity in entities:
        if hasattr(entity, 'label') and entity.label() in ['ORGANIZATION', 'PERSON', 'GPE'] and not any([hasattr(child, 'label') for child in entity]):
            main_entities.append(' '.join([word for word, tag in entity.leaves()]))
    return main_entities

def lemmetize(user_input):
    # Lemmatize the words in user input
    lemmatizer = WordNetLemmatizer()
    lemmatized_input = [lemmatizer.lemmatize(word) for word in word_tokenize(user_input)]
    return lemmatized_input

# Answers Module
We managed to make the chatbot answer a dynamic answer according the user intents by getting the intents and comparing it to the answers csv file.

In [44]:
def get_output(intents, sentiment):
    sent_output = ""
    output = ""
    if sentiment["compound"] <= -0.5:
        sent_output += "I'm sorry to hear that you're feeling negative..."
    elif sentiment["compound"] >= 0.5:
        sent_output += "I'm glad to hear that you're feeling positive!."

    prime = str.lower(intents[0][0])
    sub = str.lower(intents[0][1])
    dataframe_answers = pd.read_csv("answers.csv")

    # defining the possible answers from the csv file for each intent.
    hotel_answer = dataframe_answers.loc[
        (dataframe_answers["primary"] == "book") & (dataframe_answers["sub"] == "hotel")]
    hotel_answer_list = hotel_answer.values.tolist()

    car_answer = dataframe_answers.loc[(dataframe_answers["primary"] == "book") & (dataframe_answers["sub"] == "car")]
    car_answer_list = car_answer.values.tolist()

    flight_answer = dataframe_answers.loc[
        (dataframe_answers["primary"] == "book") & (dataframe_answers["sub"] == "flight")]
    flight_answer_list = flight_answer.values.tolist()

    package_answer = dataframe_answers.loc[
        (dataframe_answers["primary"] == "book") & (dataframe_answers["sub"] == "package")]
    package_answer_list = package_answer.values.tolist()

    change_answer = dataframe_answers.loc[(dataframe_answers["primary"] == "change")]
    change_answer_list = change_answer.values.tolist()

    cancel_answer = dataframe_answers.loc[(dataframe_answers["primary"] == "cancel")]
    cancel_answer_list = cancel_answer.values.tolist()

    exit_answer = dataframe_answers.loc[(dataframe_answers["primary"] == "exit")]
    exit_answer_list = exit_answer.values.tolist()

    if prime == 'book':
        if sub == 'hotel':
            answer = random.choice(hotel_answer_list[2:])
            output = answer[2]
        elif sub == "car":
            answer = random.choice(car_answer_list[2:])
            output = answer[2]

        elif sub == "flight":
            answer = random.choice(flight_answer_list[2:])
            output = answer[2]

        elif sub == "package":
            answer = random.choice(package_answer_list[2:])
            output = answer[2]

        else:
            output += " I'm sorry, I didn't understand your request."
    elif prime == 'change':
        answer = random.choice(change_answer_list[2:])
        output = answer[2]

    elif prime == 'cancel':
        answer = random.choice(cancel_answer_list[2:])
        output = answer[2]

    elif prime == 'exit':
        answer = random.choice(exit_answer_list[2:])
        output = answer[2]

    else:
        output += " I'm sorry, I didn't understand your request."
    return sent_output + output

# Side processes for booking hotel, flight and car rental.
## Preparing csv file of bookings:

In [45]:
#Reading the bookings file for making changes.
def read_bookings_from_csv(csv_file_path):
    with open(csv_file_path, 'r', newline='') as f:
        reader = csv.DictReader(f)
        bookings_dict = {}
        for row in reader:
            booking_id = (row['id'])
            booking_type = row['type']
            location = row['location']
            start_date = row['start_date']
            end_date = row['end_date']
            bookings_dict[booking_id] = {'type': booking_type, 'location': location, 'start_date': start_date, 'end_date': end_date}
    return bookings_dict
# read in the bookings from the CSV file
bookings_dict = read_bookings_from_csv("bookings.csv")

# Writing saved data to the csv file (changed location/dates)
def write_bookings_to_csv(bookings_dict, csv_file_path):
    with open(csv_file_path, 'w', newline='') as f:
        fieldnames = ['id','name', 'type', 'location', 'start_date', 'end_date']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for booking_id, booking in bookings_dict.items():
            writer.writerow({'id': booking_id, 'type': booking['type'], 'location': booking['location'], 'start_date': booking['start_date'], 'end_date': booking['end_date']})

#Saving new booking to csv file.
def save_booking_to_csv(new_booking, csv_file_path):
    # Create a new ID for the booking
    with open(csv_file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        # Write the new booking as a row in the CSV file
        writer.writerow([new_booking['id'], new_booking['name'], new_booking['type'], new_booking['location'], new_booking['start_date'], new_booking['end_date']])
    print("Booking saved with ID:", new_booking['id'])

#Getting the type of booking from the csv file.
def get_booking_type_from_csv(csv_file_path, booking_id):
    with open(csv_file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            if row[0] == booking_id:
                return row[1]  #  booking type is in second column
    return None

## Business Processes (Booking hotel/flight/package, renting car)


In [46]:
# function to book a hotel
def book_hotel(user_name,main_state):
    hotel_location = get_location("hotel",main_state)
    if hotel_location == "exit":
        return
    hotel_start_date, hotel_end_date = get_dates("hotel")

    # Code to book a hotel room goes here
    total_days = day_count(hotel_start_date,hotel_end_date)
    print("Booking hotel in", hotel_location, "for", hotel_start_date, "until",hotel_end_date, ". Total Days: ", total_days )

    booking_id = str(uuid.uuid4())
    new_booking = {'id': booking_id, 'name':user_name, 'type': 'hotel', 'location': hotel_location, 'start_date': hotel_start_date, 'end_date': hotel_end_date}
    bookings_dict[booking_id] = new_booking

    # Save booking to CSV
    save_booking_to_csv(new_booking, 'bookings.csv')

# Define function to book a car
def book_car(user_name,main_state):
    car_location = get_location("car",main_state)
    if car_location == "exit":
        return
    car_start_date, car_end_date = get_dates("car rental")

    total_days = day_count(car_start_date,car_end_date)
    print("Booking car in", car_location, "for", car_start_date, "until",car_end_date, " .Total Days: ", total_days)

    booking_id = str(uuid.uuid4())
    new_booking = {'id': booking_id, 'name':user_name, 'type': 'car', 'location': car_location, 'start_date': car_start_date, 'end_date': car_end_date}
    bookings_dict[booking_id] = new_booking

    save_booking_to_csv(new_booking, 'bookings.csv')

# function to book a flight
def book_flight(user_name,main_state):
    flight_destination = get_location("flight",main_state)
    if flight_destination == "exit":
        return
    flight_start_date, flight_end_date = get_dates("flight")

    total_days = day_count(flight_start_date,flight_end_date)
    print("Booking flight to", flight_destination, "for", flight_start_date, "until",flight_end_date, " .Total Days: ", total_days)

    booking_id = str(uuid.uuid4())
    new_booking = {'id': booking_id, 'name':user_name , 'type': 'flight', 'location': flight_destination, 'start_date': flight_start_date, 'end_date': flight_end_date}
    bookings_dict[booking_id] = new_booking

    # Save booking to CSV
    save_booking_to_csv(new_booking, 'bookings.csv')

# Define function to book a flight
def book_package(user_name,main_state):
    package_destination = get_location("package",main_state)
    if package_destination == "exit":
        return
    package_start_date, package_end_date = get_dates("package")

    total_days = day_count(package_start_date,package_end_date)
    print("Booking package to", package_destination, "for", package_start_date, "until",package_end_date, " .Total Days: ", total_days)

    booking_id = str(uuid.uuid4())
    new_booking = {'id': booking_id, 'name':user_name , 'type': 'flight', 'location': package_destination, 'start_date': package_start_date, 'end_date': package_end_date}
    bookings_dict[booking_id] = new_booking

    # Save booking to CSV
    save_booking_to_csv(new_booking, 'bookings.csv')

## Essentials (getting dates, location, counting days)

In [47]:
# Get the dates from the user.
def get_dates(booking_type):
    current_state = "Get dates"
    while True:
        start_date_input = input(f"When do you want to start your {booking_type} booking? (Please enter in YYYY-MM-DD format): ")
        if if_exit(start_date_input):
            exit()
        end_date_input = input(f"When do you want to end your {booking_type} booking? (Please enter in YYYY-MM-DD format): ")
        if if_exit(end_date_input):
            exit()
        date_pattern = re.compile("^\d{4}-\d{2}-\d{2}$")
        if not date_pattern.match(start_date_input) or not date_pattern.match(end_date_input):
            print("Invalid date format. Please enter in YYYY-MM-DD format.")
        else:
            return start_date_input, end_date_input

# Counting the days of the booking.
def day_count(start_date, end_date):
    try:
        start_year, start_month, start_day = map(lambda x: int(x.lstrip("0")), start_date.split("-"))
        end_year, end_month, end_day = map(lambda x: int(x.lstrip("0")), end_date.split("-"))
        start_date = date(start_year, start_month, start_day)
        end_date = date(end_year, end_month, end_day)
        delta = end_date - start_date
        return delta.days
    except ValueError:
        print("Invalid date format. Please enter dates in the format YYYY-MM-DD.")
        return None

#getting the location from the user.
def get_location(intent,main_state):
    current_state = "Get Location"
    location =""
    if intent == "hotel":
        location = input("Where would you like to book a hotel room? ")
        process_chat_input(location,current_state,main_state)
        if if_exit(location):
            location = "exit"
    elif intent == "car":
        location = input("Where would you like to rent a car? ")
        if if_exit(location):
            exit()
    elif intent == "flight":
        location = input("Where would you like to fly to? ")
        process_chat_input(location,current_state,main_state)
        if if_exit(location):
            location = "exit"
    elif intent == "package":
        location = input("Where would you like to order vacation package? ")
        process_chat_input(location,current_state,main_state)
        if if_exit(location):
            location = "exit"

    flag = True
    while flag:
        if not is_location_valid(location):
            location = input("Wrong Location, Try Again:")
            process_chat_input(location,current_state,main_state)
            if if_exit(location):
                location = "exit"
        else:
            print("Location founded!")
            flag = False

    return location

# Getting the booking ID.
def get_booking_id(main_state):
    current_state = "Get ID"
    while True:
        booking_id = input("Please enter the booking ID: ")
        process_chat_input(booking_id, current_state,main_state)
        if if_exit(booking_id) or booking_id == "exit":
            booking_id = "exit"
            return booking_id
        if booking_id.isalnum():
            if int(booking_id) not in bookings_dict:
                print("Invalid booking ID.")
            else:
                break
    return booking_id

## Managing bookings: change and cancel

In [48]:
# Changing booking - location/dates.
def change_booking(csv_file_path,main_state):
    current_state = "Change"
    bookings_dict = read_bookings_from_csv(csv_file_path)
    booking_id = get_booking_id(main_state)
    if booking_id == "exit":
        return
    while True:
        attribute = input("Which attribute would you like to change? (location, dates): ")
        if if_exit(attribute):
            exit()
        process_chat_input(attribute,current_state,main_state)
        choise = classify_input(attribute)
        intent = get_booking_type_from_csv("bookings.csv", booking_id)
        for intents in choise:
            if intents[0]!="change":
                print("Invalid attribute.")
            elif intents[0]=="exit":
                return
            if intents[1] == "location":
                new_value = get_location(intent,main_state)
                bookings_dict[int(booking_id)][attribute.replace(" ", "_")] = new_value
                break
            elif intents[1] == "date":
                new_value = get_dates(intent)
                bookings_dict[int(booking_id)][attribute.replace(" ", "_")] = new_value
                break
            active_learning(intents,attribute)

        write_bookings_to_csv(bookings_dict, csv_file_path)
        print("Booking", booking_id, "has been updated.")

#  Canceling booking and deleting it from data.
def cancel_booking(csv_file_path,main_state):
    current_state = "Cancel"
    cancel = input("Are you sure you want to cancel your booking? Yes/No")
    if str.lower(cancel) == "no":
        return
    bookings_dict = read_bookings_from_csv(csv_file_path)
    booking_id = get_booking_id(main_state)
    if booking_id == "exit":
        return
    del bookings_dict[int(booking_id)]
    write_bookings_to_csv(bookings_dict, csv_file_path)
    print("Booking", booking_id, "has been canceled. No Refund!!!")

# API to location service

In [49]:
# Using API to check if the location exist.
def is_location_valid(location):
    url = f"https://nominatim.openstreetmap.org/search?q={location}&format=json"
    response = requests.get(url)
    json_data = response.json()
    if len(json_data) == 0:
        return False
    else:
        return True

# EXTRAS: Active learning feature

In [50]:
# Asking the user if the identification was correct, if it does, adding it and the intents to the data.
def active_learning(intents,user_input):
    confirm_intents = input(f"Did I identify your intent(s) correctly? ({intents}) (yes/no): ")
    if confirm_intents.lower() in ["y", "yes"]:
        # Append the input, primary intent, and sub-intent to data.csv
        with open('data.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([user_input, intents[0][0], intents[0][1]])

# Online Module


In [51]:
# Getting user's name.
def get_name():
    current_state = "Get name"
    print("Hi! What's your name?")
    user_name = input("Enter your name: ")
    if if_exit(user_name):
        exit()
    print(f"Hi {user_name}, I'm so glad to talk to you and to be at your service.")
    return user_name

#The main function for the online chatting with navigating between states according the intents.
def run_online_chatbot():
    main_state = "online"
    current_state = "Get intent"
    user_name = get_name()
    run = True
    while run:
        user_input = input(f"What can I help you with today, {user_name}? ")
        output, intents, sentiment, entities, lemmatized_input, next_state = process_chat_input(user_input, current_state,main_state)
        if str.lower(user_input)== 'exit' or if_exit(user_input):
            return
        # Based on the intent, prompt the user for additional information
        for intent in intents:
            if str.lower(intent[0]) == 'book':
                if str.lower(intent[1]) == 'hotel':
                    book_hotel(user_name,main_state)
                elif str.lower(intent[1]) == "car":
                    book_car(user_name,main_state)
                elif str.lower(intent[1]) == "flight":
                    book_flight(user_name,main_state)
                elif str.lower(intent[1]) == "package":
                    book_package(user_name,main_state)
            elif str.lower(intent[0]) == 'change':
                change_booking("bookings.csv",main_state)
            elif str.lower(intent[0]) == 'cancel':
                cancel_booking("bookings.csv",main_state)
        active_learning(intents,user_input)

    continue_conv = input("Anything else? (yes/no): ")
    if continue_conv.lower() in ["y", "yes"]:
        run_online_chatbot()
    else:
        choose_state()

In [52]:
# Processing each user's input to print the identified values of intents, sentiments, etc.
def process_chat_input(user_input,current_state,main_state):
    # Check if input is a string
    if not isinstance(user_input, str):
        return "", [], "", [], "", ""

    intents = classify_input(user_input)
    sentiment = track_sentiment(user_input)
    entities = extract_entities(user_input)
    lemmatized_input = lemmetize(user_input)
    output = get_output(intents, sentiment)
    output_path = "online_outputs.csv"
    next_state = get_next_state(intents,current_state)

    if main_state == "online":
        # Write output to file
        with open(output_path, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([user_input, output, intents, sentiment, entities, lemmatized_input, next_state])

    print_data(user_input, output, intents, sentiment, entities, lemmatized_input,next_state)

    # Update current state with next state
    current_state = next_state

    return output, intents, sentiment, entities, lemmatized_input, next_state

# printing the outputs of the identified parameters.
def print_data(user_input, output, intents, sentiment, entities, lemmatized_input,next_state):
    print('\nUser Input: ', user_input,
          "\nLemmatized Input: ", lemmatized_input,
          "\nIntent: ", intents,
          "\nSentiment: ", sentiment,
          "\nEntities: ", entities,
          "\nChatbot Output: ", output,
          "\nNext state: ",next_state)

# Essential function - if the user's intent is to exit, it will exit.
def if_exit(user_input):
    intents = classify_input(user_input)
    if str.lower(user_input) == "exit":
        return True
    for intent in intents:
        if intent[0] == "exit":
            return True
        else:
            return False

# If-Else function to define what is the next state of the chatbot.
def get_next_state(intents,current_state):
    next_state = None

    if current_state == "Get name":
        next_state = "Get intent"
    if current_state == "offline":
        next_state = "Get intent"

    if current_state == "Book Hotel":
        next_state = "Get Location"
    elif current_state == "Book car":
        next_state = "Get Location"
    elif current_state == "Book flight":
        next_state = "Get Location"
    elif current_state == "Book Package":
        next_state = "Get Location"

    elif current_state == "Get Location":
        next_state = "Get dates"
    elif current_state == "Get dates":
        next_state = "Get intent"

    elif current_state == "Change":
        next_state = "Get ID"
        for intent in intents:
            if intent[1]== "location":
                next_state = "Get Location"
            if intent[1] == "date":
                next_state = "Get dates"

    elif current_state == "Get ID":
        next_state = "Get intent"
    elif current_state == "cancel":
        next_state = "Get ID"
    return next_state

# Offline Module


In [53]:
# Main function to run the chatbot offline, and saving the outputs to csv file.
def run_offline_chatbot():
    main_state = "offline"
    current_state = "offline"
    chat_input_df = pd.read_csv("script.csv")
    chat_output_df = pd.DataFrame(columns=['text', 'lemmatized', 'output', 'intent', 'entities', 'polarity', 'next_state'])

    for index, row in chat_input_df.iterrows():
        text = row['text']
        output, intents, sentiment, entities, lemmatized_input, next_state = process_chat_input(text, current_state,main_state)
        chat_output_df = pd.concat([chat_output_df, pd.DataFrame({'text': text, 'lemmatized': str(lemmatized_input),
                                                                  'output': str(output), 'intent': str(intents),
                                                                  'entities': str(entities), 'polarity': str(sentiment),
                                                                  'next_state': str(next_state)}, index=[0])], ignore_index=True)
        if next_state:
            current_state = next_state
    chat_output_df.to_csv("offline_outputs.csv", index=False)
    return chat_output_df

In [54]:
run_chat()

Chatbot is now in Online mode.
Hi! What's your name?
Hi Ben, I'm so glad to talk to you and to be at your service.

User Input:  I want to book hotel 
Lemmatized Input:  ['I', 'want', 'to', 'book', 'hotel'] 
Intent:  [['book', 'hotel']] 
Sentiment:  {'neg': 0.0, 'neu': 0.698, 'pos': 0.302, 'compound': 0.0772} 
Entities:  [] 
Chatbot Output:  It's amazing! Allow us to locate you suitable accomodation. 
Next state:  None

User Input:  Tokyo 
Lemmatized Input:  ['Tokyo'] 
Intent:  [['book', 'Flight']] 
Sentiment:  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 
Entities:  ['Tokyo'] 
Chatbot Output:  Let's buy a flight ticket! 
Next state:  Get dates
Location founded!
Booking hotel in Tokyo for 2023-12-12 until 2023-12-13 . Total Days:  1
Booking saved with ID: e5fc120d-c22d-4132-aada-1aee2ab48c57

User Input:  exit 
Lemmatized Input:  ['exit'] 
Intent:  [['exit', 'none']] 
Sentiment:  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 
Entities:  [] 
Chatbot Output:  We are grat